In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

## Run model on clean dataset

In [ ]:
df = pd.read_pickle("./pickles/df_small_clean.pkl")
df.describe()

In [ ]:
# y = df["position"]
# # X = df.drop("position", axis=1).copy()
# X = df.copy()

y = df["importance"]
X = df.drop(["importance","click_bool","booking_bool"], axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [ ]:
rfc = RandomForestClassifier(n_jobs=1)
model = rfc.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
# make a solution
X_test['predicted_importance'] = predictions
print(X_test['predicted_importance'].unique())
# X_test.head(25)

X_test.groupby('srch_id').apply(lambda x: x.sort_values(["predicted_importance"], ascending = False)).reset_index(drop=True)
# X_test.head(25)


In [ ]:
print(np.arange(1,10+1))

In [ ]:
# oke predicted_importance kolom is nu gesorteerd, dus nu predicted_rank toevoegen
for srchid, group in X_test.groupby('srch_id'):
    length = len(group['predicted_importance'])
    group['predicted_rank'] = np.arange(1,length+1)

# nu de positions, click_bool en book_bool weer terughalen
X_test['position'] = pos
X_test['click_bool'] = click
X_test['book_bool'] = book

# en picklen
X_test.to_pickle("./pickles/random_forest_small_solution.pkl")

In [ ]:
pred_groups = X_test.groupby('srch_id').agg({'predicted_values':lambda x: list(x)})
print(pred_groups)

In [ ]:
X_test['predicted_rank'] = predictions
X_test['position'] = y
X_test.to_pickle("./pickles/random_forest_small_solution.pkl")

In [ ]:
# convert to numpy array
y = df['prop_id'].to_numpy()
X = df.drop("prop_id", axis=1).copy().to_numpy()

In [ ]:
@jit
def rand_for(X_train, y_train, X_test):
    rfc = RandomForestClassifier(n_jobs=1)
    model = rfc.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    return predictions

    
df = pd.read_pickle("./pickles/df_small_clean.pkl")
y = df["position"]
# X = df.drop("position", axis=1).copy()
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_test.drop("position", axis=1)
X_test['position'] = 0

numpy_result = rand_for(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy())

#df = pd.DataFrame(data=numpy_result, index=df['srch_id'], columns=["prop_id", "srch_id", "position", "price_quality", "competitor_bool", "availability_bools", "visited_before", "click_bool", "booking_bool", "promotion_flag", "prop_brand_bool", "random_bool"])

X_test['predicted_rank'] = numpy_result
X_test['position'] = y
X_test.to_pickle("./pickles/random_forest_small_solution.pkl")